# Washington DOT Traffic API

In [8]:
import pandas as pd

import requests

## Import env variables

In [10]:
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

# access the env variables
api_key = os.getenv("API_KEY")
db_url = os.getenv("DATABASE_URL")

# print to verify
# print(f"API Key: {api_key}")
# print(f"Database URL: {db_url}")

## Get API data

In [6]:
full_url = f"{db_url}{api_key}"

response = requests.get(full_url)

# check if request was successful
if response.status_code == 200:
    print("Data fetched successfully.")
    data = response.json()
    df = pd.DataFrame(data)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Data fetched successfully.


In [7]:
df.head()

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,25,25,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1737524700000-0800)/,2
1,26,28,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1737524700000-0800)/,3
2,26,28,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1737524700000-0800)/,4
3,9,9,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1737524700000-0800)/,5
4,9,9,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1737524700000-0800)/,6


In [9]:
df.shape

(168, 9)